# labeling 2
# 중요 !! : 메이저는 따로 A00_07을 돌려야함!
### 1. 백테스팅할때 방식처럼 1번 매수하면 그사이 매수는 무시하고 다음 sell로 넘어가는 1회,1회 매매하는대로 라벨을 선정
### 2. 1일 뒤를 기준으로 라벨을 삼는 Future_label 생성
- Future_label이 모델학습시 찐라벨이 됨. 실제보다 하루 앞당겨져있음.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

def create_optimized_labels(df, window=5):
    """
    백테스터와 동일한 로직으로 라벨 생성 + 타이밍 최적화
    
    Parameters:
    -----------
    df : pd.DataFrame
        원본 데이터 (Close, Label 컬럼 필요)
    window : int
        타이밍 최적화 윈도우 (앞뒤 며칠)
    
    Returns:
    --------
    pd.DataFrame : 최적화된 라벨이 추가된 데이터
    """
    
    result_df = df.copy()
    n = len(df)
    
    # 원본 라벨
    original_labels = df['Label'].values
    
    # 1단계: 백테스터와 동일한 로직으로 유효한 신호만 추출
    valid_signals = []  # (index, signal_type) 형태로 저장
    current_position = 'CASH'  # CASH 또는 LONG
    
    # print(f"백테스터 로직으로 유효 신호 추출 중...")
    
    for i in range(n):
        current_label = original_labels[i]
        
        # 백테스터와 동일한 매매 조건
        if current_label == 2 and current_position == 'CASH':  # Buy 가능
            valid_signals.append((i, 'BUY'))
            current_position = 'LONG'
            
        elif current_label == 0 and current_position == 'LONG':  # Sell 가능
            valid_signals.append((i, 'SELL'))
            current_position = 'CASH'
    
    buy_signals = [idx for idx, signal in valid_signals if signal == 'BUY']
    sell_signals = [idx for idx, signal in valid_signals if signal == 'SELL']
    
    # print(f"원본 신호: Buy {sum(original_labels == 2)}개, Sell {sum(original_labels == 0)}개")
    # print(f"유효 신호: Buy {len(buy_signals)}개, Sell {len(sell_signals)}개")
    
    # 2단계: 타이밍 최적화
    optimized_buy_signals = []
    optimized_sell_signals = []
    
    # Buy 신호 최적화 (앞뒤 window일 중 가장 낮은 가격일)
    for buy_idx in buy_signals:
        start_idx = max(0, buy_idx - window)
        end_idx = min(n - 1, buy_idx + window)
        
        # 해당 구간의 종가들과 인덱스들
        window_section = df.iloc[start_idx:end_idx + 1]
        min_price_idx = window_section['Close'].idxmin()
        
        # DataFrame 인덱스를 정수 인덱스로 변환
        best_buy_idx = df.index.get_loc(min_price_idx)
        optimized_buy_signals.append(best_buy_idx)
    
    # Sell 신호 최적화 (앞뒤 window일 중 가장 높은 가격일)
    for sell_idx in sell_signals:
        start_idx = max(0, sell_idx - window)
        end_idx = min(n - 1, sell_idx + window)
        
        # 해당 구간의 종가들과 인덱스들
        window_section = df.iloc[start_idx:end_idx + 1]
        max_price_idx = window_section['Close'].idxmax()
        
        # DataFrame 인덱스를 정수 인덱스로 변환
        best_sell_idx = df.index.get_loc(max_price_idx)
        optimized_sell_signals.append(best_sell_idx)
    
    # 3단계: 최적화된 라벨 생성
    optimized_labels = np.ones(n, dtype=int)  # 기본값: Hold(1)
    
    # Buy 신호 적용
    for buy_idx in optimized_buy_signals:
        if 0 <= buy_idx < n:
            optimized_labels[buy_idx] = 2  # Strong_Buy
    
    # Sell 신호 적용
    for sell_idx in optimized_sell_signals:
        if 0 <= sell_idx < n:
            optimized_labels[sell_idx] = 0  # Strong_Sell
    
    # 4단계: 최종 검증 (백테스터 로직 재확인)
    final_labels = np.ones(n, dtype=int)  # 기본값: Hold(1)
    current_position = 'CASH'
    
    for i in range(n):
        current_label = optimized_labels[i]
        
        if current_label == 2 and current_position == 'CASH':  # Buy 가능
            final_labels[i] = 2
            current_position = 'LONG'
            
        elif current_label == 0 and current_position == 'LONG':  # Sell 가능
            final_labels[i] = 0
            current_position = 'CASH'
        
        # 나머지는 Hold(1) 유지
    
    # 결과 저장
    result_df['Optimized_Label'] = final_labels
    
    # 라벨명 추가
    label_names = {0: 'Strong_Sell', 1: 'Hold', 2: 'Strong_Buy'}
    result_df['Optimized_Label_Name'] = result_df['Optimized_Label'].map(label_names)
    
    # 최종 통계
    final_buy_count = sum(final_labels == 2)
    final_sell_count = sum(final_labels == 0)
    final_hold_count = sum(final_labels == 1)
    
    # print(f"최종 라벨: Buy {final_buy_count}개, Sell {final_sell_count}개, Hold {final_hold_count}개")
    # print(f"Buy-Sell 균형: {min(final_buy_count, final_sell_count)}/{max(final_buy_count, final_sell_count)}")
    
    # 연속 신호 검증
    consecutive_issues = 0
    for i in range(1, n):
        if final_labels[i-1] == 2 and final_labels[i] == 2:  # 연속 Buy
            consecutive_issues += 1
        elif final_labels[i-1] == 0 and final_labels[i] == 0:  # 연속 Sell
            consecutive_issues += 1
    
    if consecutive_issues == 0:
        # print("✅ 연속 신호 없음 - 백테스터 호환 완료!")
        pass
    else:
        print(f"⚠️ 연속 신호 {consecutive_issues}개 발견")
    
    return result_df

def create_optimized_labels_for_all_stocks(stocks_data, window=5, save_path="optimized_stock_data"):
    """
    모든 종목에 대해 최적화된 라벨 생성
    
    Parameters:
    -----------
    stocks_data : dict
        종목별 데이터 딕셔너리
    window : int
        타이밍 최적화 윈도우
    save_path : str
        저장 경로
    
    Returns:
    --------
    dict : 최적화된 라벨이 포함된 종목 데이터
    """
    
    # 저장 경로 생성
    os.makedirs(save_path, exist_ok=True)
    
    optimized_stocks = {}
    
    print(f"최적화된 라벨 생성 중... (윈도우: {window}일)")
    print("="*60)
    
    for symbol in tqdm(stocks_data.keys(), desc="라벨 최적화"):
        try:
            df = stocks_data[symbol].copy()
            
            # 필수 컬럼 확인
            if 'Close' not in df.columns or 'Label' not in df.columns:
                print(f"⚠️ {symbol}: 필수 컬럼 누락 (Close, Label)")
                continue
            
            print(f"\n처리 중: {symbol}")
            
            # 최적화된 라벨 생성
            optimized_df = create_optimized_labels(df, window=window)
            
            # 결과 저장
            optimized_stocks[symbol] = optimized_df
            
            # CSV 저장
            optimized_df.to_csv(f"{save_path}/{symbol}_optimized.csv")
            
            print(f"✅ {symbol}: 저장 완료")
            
        except Exception as e:
            print(f"❌ {symbol}: 오류 발생 - {e}")
            continue
    
    print(f"\n전체 최적화 완료!")
    print(f"성공: {len(optimized_stocks)}개")
    print(f"저장 경로: {save_path}")
    
    return optimized_stocks

def compare_label_performance(df, original_col='Label', optimized_col='Optimized_Label'):
    """
    원본 라벨 vs 최적화된 라벨 성능 비교
    
    Parameters:
    -----------
    df : pd.DataFrame
        두 라벨이 모두 있는 데이터
    original_col : str
        원본 라벨 컬럼명
    optimized_col : str
        최적화된 라벨 컬럼명
    
    Returns:
    --------
    dict : 비교 결과
    """
    
    result = {}
    
    # 라벨 분포 비교
    original_dist = df[original_col].value_counts().sort_index()
    optimized_dist = df[optimized_col].value_counts().sort_index()
    
    print("라벨 분포 비교:")
    print(f"{'라벨':<15} {'원본':<10} {'최적화':<10} {'변화':<10}")
    print("-" * 45)
    
    for label in [0, 1, 2]:
        orig_count = original_dist.get(label, 0)
        opt_count = optimized_dist.get(label, 0)
        change = opt_count - orig_count
        
        label_name = {0: 'Strong_Sell', 1: 'Hold', 2: 'Strong_Buy'}[label]
        print(f"{label_name:<15} {orig_count:<10} {opt_count:<10} {change:+d}")
    
    # 신호 간격 분석 (Buy-Sell 패턴)
    def analyze_signal_pattern(labels):
        buy_indices = np.where(labels == 2)[0]
        sell_indices = np.where(labels == 0)[0]
        
        if len(buy_indices) == 0 or len(sell_indices) == 0:
            return {'avg_hold_period': 0, 'num_complete_trades': 0}
        
        # 완전한 매매 사이클 찾기
        complete_trades = 0
        hold_periods = []
        
        current_position = 'CASH'
        last_buy_idx = None
        
        for i in range(len(labels)):
            if labels[i] == 2 and current_position == 'CASH':  # Buy
                last_buy_idx = i
                current_position = 'LONG'
            elif labels[i] == 0 and current_position == 'LONG':  # Sell
                if last_buy_idx is not None:
                    hold_period = i - last_buy_idx
                    hold_periods.append(hold_period)
                    complete_trades += 1
                current_position = 'CASH'
        
        avg_hold_period = np.mean(hold_periods) if hold_periods else 0
        
        return {
            'avg_hold_period': avg_hold_period,
            'num_complete_trades': complete_trades
        }
    
    # 패턴 분석
    original_pattern = analyze_signal_pattern(df[original_col].values)
    optimized_pattern = analyze_signal_pattern(df[optimized_col].values)
    
    print(f"\n매매 패턴 분석:")
    print(f"{'지표':<20} {'원본':<15} {'최적화':<15}")
    print("-" * 50)
    print(f"{'완전 매매 횟수':<20} {original_pattern['num_complete_trades']:<15} {optimized_pattern['num_complete_trades']:<15}")
    print(f"{'평균 보유기간':<20} {original_pattern['avg_hold_period']:.1f}일{'':<8} {optimized_pattern['avg_hold_period']:.1f}일")
    
    return {
        'original_distribution': original_dist.to_dict(),
        'optimized_distribution': optimized_dist.to_dict(),
        'original_pattern': original_pattern,
        'optimized_pattern': optimized_pattern
    }

# 메인 실행 함수
def process_stocks_with_optimized_labels(input_path, output_path, window=5, test_symbols=None):
    """
    기존 CSV 파일들을 읽어서 최적화된 라벨을 적용하고 새로운 경로에 저장

    """
    
    import glob
    
    print(f"📁 입력 경로: {input_path}")
    print(f"📁 출력 경로: {output_path}")
    print(f"🔧 최적화 윈도우: {window}일")
    print("="*80)
    
    # 다양한 CSV 파일 패턴 시도
    patterns = [
        "*_enhanced.csv",    # enhanced 파일들
        "*_cleaned.csv",     # cleaned 파일들  
        "*.csv"              # 모든 CSV 파일들
    ]
    
    csv_files = []
    used_pattern = None
    
    for pattern in patterns:
        files = glob.glob(os.path.join(input_path, pattern))
        if files:
            csv_files = files
            used_pattern = pattern
            print(f"✅ 파일 패턴 발견: {pattern} ({len(files)}개 파일)")
            break
    
    if not csv_files:
        print(f"❌ CSV 파일을 찾을 수 없습니다: {input_path}")
        print(f"   시도한 패턴: {', '.join(patterns)}")
        return None
    
    # 출력 경로 생성
    os.makedirs(output_path, exist_ok=True)
    print(f"📂 출력 디렉토리 생성/확인: {output_path}")
    
    # 데이터 로딩
    stocks_data = {}
    
    print(f"\n📊 CSV 파일 로딩 중...")
    for file_path in tqdm(csv_files, desc="파일 로딩"):
        filename = os.path.basename(file_path)
        
        # 종목명 추출 (다양한 접미사 제거)
        symbol = filename.replace('.csv', '')
        for suffix in ['_enhanced', '_cleaned', '_processed', '_features']:
            symbol = symbol.replace(suffix, '')
        
        # 특정 종목만 처리하는 경우
        if test_symbols and symbol not in test_symbols:
            continue
        
        try:
            # CSV 읽기 (첫 번째 컬럼을 날짜 인덱스로)
            df = pd.read_csv(file_path)
            
            # 날짜 인덱스 설정
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df.set_index('Date', inplace=True)
            elif 'Unnamed: 0' in df.columns:
                df.index = pd.to_datetime(df['Unnamed: 0'])
                df.drop('Unnamed: 0', axis=1, inplace=True)
            else:
                df.index = pd.to_datetime(df.index)
            
            # 필수 컬럼 확인
            if 'Close' not in df.columns:
                print(f"⚠️ {symbol}: Close 컬럼 없음")
                continue
                
            if 'Label' not in df.columns:
                print(f"⚠️ {symbol}: Label 컬럼 없음")
                continue
            
            stocks_data[symbol] = df
            
        except Exception as e:
            print(f"❌ 파일 로딩 실패: {filename} - {e}")
            continue
    
    print(f"✅ 로딩 완료: {len(stocks_data)}개 종목")
    
    if len(stocks_data) == 0:
        print("❌ 유효한 데이터가 없습니다.")
        return None
    
    # 최적화된 라벨 생성 및 저장
    print(f"\n🔄 최적화된 라벨 생성 중...")
    optimized_stocks = {}
    
    for symbol in tqdm(stocks_data.keys(), desc="라벨 최적화"):
        try:
            df = stocks_data[symbol].copy()
            
            print(f"\n🔍 처리 중: {symbol} ({len(df)}행)")
            
            # 최적화된 라벨 생성
            optimized_df = create_optimized_labels(df, window=window)
            
            # 결과 저장
            optimized_stocks[symbol] = optimized_df
            
            # CSV 파일로 저장 (원본 파일명 유지하되 경로만 변경)
            output_filename = f"{symbol}_optimized.csv"
            output_file_path = os.path.join(output_path, output_filename)
            optimized_df.to_csv(output_file_path)
            
            print(f"✅ {symbol}: 저장 완료 → {output_filename}")
            
        except Exception as e:
            print(f"❌ {symbol}: 오류 발생 - {e}")
            continue
    
    # 최종 결과 요약
    print(f"\n" + "="*80)
    print(f"🎯 최적화 완료!")
    print(f"   성공: {len(optimized_stocks)}개 종목")
    print(f"   실패: {len(stocks_data) - len(optimized_stocks)}개 종목")
    print(f"   저장 경로: {output_path}")
    print(f"   파일 형식: [종목명]_optimized.csv")
    
    return optimized_stocks

# 간편 사용 함수 (기존 함수명 유지)
def process_enhanced_stocks_to_optimized(input_path, output_path, window=1, test_symbols=None):
    """기존 함수명과의 호환성을 위한 래퍼 함수"""
    return process_stocks_with_optimized_labels(input_path, output_path, window, test_symbols)

# 실행

In [2]:
# 여러분의 경로로 실행
input_path = "/workspace/AI모델/projects/coin/data/1h/labeling/00" 
output_path = "/workspace/AI모델/projects/coin/data/1h/labeling/01" 

# 전체 종목 처리
optimized_stocks = process_stocks_with_optimized_labels(
    input_path=input_path,
    output_path=output_path,
    window=1  # 앞뒤 5일 윈도우
)

# # 특정 종목만 테스트하고 싶다면
# test_stocks = process_stocks_with_optimized_labels(
#     input_path=input_path,
#     output_path=output_path,
#     window=5
#     # test_symbols=['AAPL', 'MSFT', 'GOOGL']  # 특정 종목들만
# )

📁 입력 경로: /workspace/AI모델/projects/coin/data/1h/labeling/00
📁 출력 경로: /workspace/AI모델/projects/coin/data/1h/labeling/01
🔧 최적화 윈도우: 1일
✅ 파일 패턴 발견: *_enhanced.csv (20개 파일)
📂 출력 디렉토리 생성/확인: /workspace/AI모델/projects/coin/data/1h/labeling/01

📊 CSV 파일 로딩 중...


파일 로딩:   0%|          | 0/20 [00:00<?, ?it/s]

파일 로딩: 100%|██████████| 20/20 [00:15<00:00,  1.28it/s]


✅ 로딩 완료: 20개 종목

🔄 최적화된 라벨 생성 중...


라벨 최적화:   0%|          | 0/20 [00:00<?, ?it/s]

...

✅ USDT: 저장 완료 → USDT_optimized.csv

🔍 처리 중: XLM (17299행)


라벨 최적화:  95%|█████████▌| 19/20 [01:33<00:05,  5.11s/it]

✅ XLM: 저장 완료 → XLM_optimized.csv

🔍 처리 중: XRP (17302행)


라벨 최적화: 100%|██████████| 20/20 [01:38<00:00,  4.94s/it]

✅ XRP: 저장 완료 → XRP_optimized.csv

🎯 최적화 완료!
   성공: 20개 종목
   실패: 0개 종목
   저장 경로: /workspace/AI모델/projects/coin/data/1h/labeling/01
   파일 형식: [종목명]_optimized.csv
